In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp

A07.1 - Correlation MSTs

Task: Build MSTs from correlations and analyze them

+ from the correlation matrices, build the Minimum Spanning Trees (see lecture)

+ assign the correct ticker (’AAPL’, ’AMZN’, ...) as an attribute to the nodes and calculate the 

+ compare the MST degree distributions of the 1 hour, 1 minute, Normal and One-Factor correlations

In [3]:
# Load data
cormat = np.load('./data/cormat_1h.npy')

A07.2 - Conclusions
Task: finish the analysis by drawing conclusions Answer the following questions (for both timescales):

1. Is there a ‘market mode’?

2. Which are the 5 stocks with highest degree on the 1 minute and 1 hour MST?

3. Is the degree distribution of MSTs qualitatively the same at 1 minute and 1 hour?

4. Is the degree distribution of MSTs well reproduced by the null Gaussian and One-Factor models?